# ALGOS DE RECO #

In [2]:
# on importe les differentes librairies
import pandas as pd
import seaborn as sns
import numpy as np

# on importe les différentes librairies surprise de scikit
from surprise import NormalPredictor, BaselineOnly, SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans




## SVD ##